# Extraindo informações sobre as turmas
Esse notebook tem como objetivo utilizar os datasets de matriculas e turmas para pegar gerar informações a respeito das turmas.

In [1]:
import pandas as pd
import numpy as np

In [2]:
matriculas = pd.read_csv("data/matricula-de-20172.csv", sep=";")

In [3]:
# Pegando apenas as turmas de CeT
matriculas_cet = matriculas[matriculas.id_curso == 10320810]

In [4]:
matriculas_cet.descricao.unique()

array(['INDEFERIDO', 'DESISTENCIA', 'APROVADO',
       'REPROVADO POR MÉDIA E POR FALTAS', 'APROVADO POR NOTA',
       'CANCELADO', 'REPROVADO', 'TRANCADO', 'REPROVADO POR NOTA E FALTA',
       'EXCLUIDA', 'REPROVADO POR NOTA', 'REPROVADO POR FALTAS'],
      dtype=object)

In [5]:
# Pegar apenas quem começou a pagar a materia

indesejada = ['DESISTENCIA', 'INDEFERIDO']

matriculas_cet = matriculas_cet[~matriculas_cet.descricao.isin(indesejada)]

In [6]:
# matriculas_cet[matriculas_cet.descricao == 'CANCELADO']

In [7]:
matriculas_cet.shape

(56325, 10)

In [8]:
matriculas_cet.columns

Index(['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'reposicao',
       'faltas_unidade', 'media_final', 'numero_total_faltas', 'descricao'],
      dtype='object')

In [9]:
novo = matriculas_cet.drop_duplicates(subset=['id_turma', 'discente'])
novo = novo.dropna(subset=['media_final', 'numero_total_faltas'])

In [10]:
novo.shape

(15817, 10)

In [11]:
novo.columns

Index(['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'reposicao',
       'faltas_unidade', 'media_final', 'numero_total_faltas', 'descricao'],
      dtype='object')

## Extraindo as informações estatísticas

In [12]:
# Calculando qnt_disc
qnt_discentes = novo.groupby('id_turma', as_index=False)['media_final'].count() \
                    .rename(columns={'media_final': 'qnt_discentes'})

# Calculando media_turma
media_turma = novo.groupby('id_turma', as_index=False)['media_final'].mean() \
                    .rename(columns={'media_final': 'media_turma'})

# Calculando media_faltas
media_faltas = novo.groupby('id_turma', as_index=False)['numero_total_faltas'].mean() \
                    .rename(columns={'numero_total_faltas': 'media_faltas'})

# Calculando qnt_reposicao
qnt_reposicao = novo[novo.reposicao == 't'].groupby('id_turma', as_index=False)['reposicao'].count() \
                    .rename(columns={'reposicao': 'qnt_reposicao'})

# Calculando qnt_aprovados
aprovados = ['APROVADO', 'APROVADO POR NOTA']
qnt_aprovados = novo[novo.descricao.isin(aprovados)].groupby('id_turma', as_index=False)['descricao'].count() \
                    .rename(columns={'descricao': 'qnt_aprovados'})

# Calculando qnt_reprovados
reprovado = ['REPROVADO', 'REPROVADO POR NOTA', 'REPROVADO POR FALTAS', 
             'REPROVADO POR NOTA E FALTA', 'REPROVADO POR MÉDIA E POR FALTAS']
qnt_reprovados = novo[novo.descricao.isin(reprovado)].groupby('id_turma', as_index=False)['descricao'].count() \
                    .rename(columns={'descricao': 'qnt_reprovacao'})

# Calculando qnt_trancamento
trancado = ['CANCELADO', 'EXCLUIDA', 'TRANCADO']
qnt_trancamento = novo[novo.descricao.isin(trancado)].groupby('id_turma', as_index=False)['descricao'].count() \
                    .rename(columns={'descricao': 'qnt_trancamentos'})

In [13]:
# mesclando os datasets
turmas = pd.merge(left=qnt_discentes, right=qnt_aprovados, how='outer')
turmas = pd.merge(left=turmas, right=qnt_reprovados, how='outer')
turmas = pd.merge(left=turmas, right=qnt_trancamento, how='outer')

In [14]:
# Realizando o merge com as demais informações
turma_geral = pd.merge(left=turmas, right=media_turma, how='outer')
turma_geral = pd.merge(left=turma_geral, right=media_faltas, how='outer')
turma_geral = pd.merge(left=turma_geral, right=qnt_reposicao, how='outer')

# Preenchendo os valores vazios com 0
turma_geral = turma_geral.fillna(value=0)

In [15]:
turma_geral['taxa_aprovacao'] = turma_geral.qnt_aprovados / turma_geral.qnt_discentes
turma_geral['taxa_reprovacao'] = turma_geral.qnt_reprovacao / turma_geral.qnt_discentes
turma_geral['evasao'] = turma_geral.qnt_trancamentos / turma_geral.qnt_discentes

# Ainda precisa ser feito
turma_geral['qnt_aprovados_primeira'] = 0

In [16]:
turma_geral[turma_geral.qnt_discentes > 100].head()

,id_turma,qnt_discentes,qnt_aprovados,qnt_reprovacao,qnt_trancamentos,media_turma,media_faltas,qnt_reposicao,taxa_aprovacao,taxa_reprovacao,evasao,qnt_aprovados_primeira
89,57605512,122,48.0,65.0,9.0,3.586066,6.434426,6.0,0.393443,0.532787,0.073770,0
90,57605513,133,100.0,31.0,2.0,5.321805,0.000000,5.0,0.751880,0.233083,0.015038,0
91,57605514,122,99.0,19.0,4.0,5.781148,0.000000,3.0,0.811475,0.155738,0.032787,0
94,57605518,106,61.0,36.0,9.0,4.667925,12.084906,5.0,0.575472,0.339623,0.084906,0
97,57605523,104,37.0,63.0,4.0,2.986538,3.269231,21.0,0.355769,0.605769,0.038462,0


In [17]:
turma_geral.to_csv('outputs/turmas_dados_2017_2.csv', sep=';', index=False, encoding='UTF-8')